# bike-reloot data set
## assembling a dat set for a prototype 

This notebook has the purpose of making a data set for the _bike-reloot_ project. 

Steps are the following: 
- scrape will haben bike pictures
- rename them to "stolen" and "not_stolen" with consecutive IDs

---

Inside of a loop through a list of willhaben URLs with different "&pages=1": 

Structure of the "a-tag" I want to scrap, which is one entry of a bike.

```
<a 
    href="/iad/kaufen-und-verkaufen/d/bike-discounter-angebot-kross-trans-4-0-s-28-zoll-0401-578744109/" 
    id="search-result-entry-header-578744109" 
    aria-labelledby="search-result-entry-header-578744109" 
    class="sc-ca51e2d8-0 lkgIJQ sc-deecb898-1 imWndm" 
    data-testid="search-result-entry-header-578744109">
</a>
```

Then retreive the href and save it to a dataframe, for n number of pages which lead to n number of entries (say, 600 entries).


In [1]:
import requests
from bs4 import BeautifulSoup

# DEFINE BASE URL 
# = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-4552&rows=1"
# url_fahrräder = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-radsport/fahrraeder-4552"
# url_fahrräder = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-4552?topicId=1001&srcType=feed-popular-cat&srcAdd=Fahrraeder" # url retreived with js disabled 

url_fahrräder = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-4552?isNavigation=true&isISRL=true&srcType=vertical-browse&srcAdd=4552"
#url_fahrräder = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz/fahrraeder-4552?topicId=1001&srcType=feed-popular-cat&srcAdd=Fahrraeder&rows=1"
# and &page=9 &rows=1

# MAKE REQUEST ON BASE URL 
page = requests.get(url_fahrräder)
soup = BeautifulSoup(page.content, "html.parser")

# Define a function to filter elements based on their IDs within the div with ID "skip-to-resultlist"
def find_by_id_custom(tag):
    return (
        tag.name == 'a' and 
        tag.get('id', '').startswith('search-result-entry-header') and 
        tag.has_attr('id')
    )

# FIND ALL a tags within ID
links = soup.find(id="skip-to-resultlist").find_all(find_by_id_custom)
print(len(links))
print(links[0].get('href'))



5
/iad/kaufen-und-verkaufen/d/merida-cross-xt-edition-neuzustand-2126461133/


In [13]:
import pandas as pd

list_id = [ link.get('id') for link in links]
list_href = [ link.get('href') for link in links]

df = pd.DataFrame({"id": list_id,
                   "href": list_href})

# split ID into all parts and extract the unique ID 
df['id_int'] = df['id'].str.split('-',expand=True)[4]
df['links'] = "https://www.willhaben.at"+df['href']
#df['links'][1]
df

,id,href,id_int,links
0,search-result-entry-header-772596263,/iad/kaufen-und-verkaufen/d/kinderfahrrad-7725...,772596263,https://www.willhaben.at/iad/kaufen-und-verkau...


# Extracting URL links of images of bikes

In [14]:
import requests
from bs4 import BeautifulSoup
from PIL import Image

In [15]:
import requests
from bs4 import BeautifulSoup
import time

#df['links'][1]


# TEST THE SINGLE ENTRY IMG EXTRACTION
single_page = requests.get(df['links'][0])
single_soup = BeautifulSoup(single_page.content, "html.parser")

def find_by_data_test_id_custom(tag):
    return (
        tag.name == 'img' and 
        tag.get('data-testid', '').startswith('image') and 
        tag.has_attr('alt')
    )

print(len(single_soup.find_all(find_by_data_test_id_custom)))
img_links = single_soup.find_all(find_by_data_test_id_custom)

# print(single_soup.find_all('img'))

print([ link.get('src') if link.has_attr('src') 
       else link.get('data-flickity-lazyload') 
       for link in img_links ])

print([ link.get('alt')  for link in img_links ])
#img_links



4
['https://cache.willhaben.at/mmo/3/772/596/263_-348722016.jpg', 'https://cache.willhaben.at/mmo/3/772/596/263_-1845172890.jpg', 'https://cache.willhaben.at/mmo/3/772/596/263_906489552.jpg', 'https://cache.willhaben.at/mmo/3/772/596/263_-186325535.jpg']
['Bild 1 von 4', 'Bild 2 von 4', 'Bild 3 von 4', 'Bild 4 von 4']


In [16]:

# PERFORM IMAGE EXTRACTION ON WHOLE DF
img_list = {}
for i in df.index:
    time.sleep(0.1)
    #requests.get(entry)
    pass_soup = BeautifulSoup(requests.get(df['links'][i]).content, "html.parser")
    pass_image_links = pass_soup.find_all(find_by_data_test_id_custom)
    
    pass_image_src = [ link.get('src') if link.has_attr('src') 
       else link.get('data-flickity-lazyload') 
       for link in img_links ]

    pass_image_alt = [ link.get('alt') for link in pass_image_links]
    
    if len(pass_image_links) > 1: 
        img_list[df['id_int'][i]] =  {'src':pass_image_src} 
        
img_list


{'772596263': {'src': ['https://cache.willhaben.at/mmo/3/772/596/263_-348722016.jpg',
   'https://cache.willhaben.at/mmo/3/772/596/263_-1845172890.jpg',
   'https://cache.willhaben.at/mmo/3/772/596/263_906489552.jpg',
   'https://cache.willhaben.at/mmo/3/772/596/263_-186325535.jpg']}}

In [17]:
# TAKE DICT AND MAKE IT TO DF
df_src = pd.DataFrame(img_list).T.explode('src').reset_index(names='id')
df_src

,id,src
0,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...
1,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...
2,772596263,https://cache.willhaben.at/mmo/3/772/596/263_9...
3,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...


In [18]:
# GENERATE A COLUMN WITH A PREFIX
import numpy as np

df_src['label'] = df_src.groupby('id')['id'].transform(
    lambda group: np.random.choice(['stolen', 'not_stolen'], size=len(group)) 
    )
df_src

,id,src,label
0,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...,not_stolen
1,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...,stolen
2,772596263,https://cache.willhaben.at/mmo/3/772/596/263_9...,stolen
3,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...,not_stolen


In [19]:
# generate a column with a unique ID, not group ID

from pathlib import Path

df_src['src_id'] = [ Path(df_src['src'][entry]).stem.split('-')[-1] for entry in df_src.index ]
df_src.head()

,id,src,label,src_id
0,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...,not_stolen,348722016
1,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...,stolen,1845172890
2,772596263,https://cache.willhaben.at/mmo/3/772/596/263_9...,stolen,263_906489552
3,772596263,https://cache.willhaben.at/mmo/3/772/596/263_-...,not_stolen,186325535


# Download all the pictures
Now, it is essential to find a way to rename the figures like: "stolen_78999345" or "not_stolen_5643345"

In [20]:
import io
import requests
from PIL import Image
from pathlib import Path

# Create a byte object out of image_content and store it in the variable image_file
image_file = io.BytesIO(requests.get(df_src['src'][0]).content)

image = Image.open(image_file).convert("RGB")

# assuming there is a folder "figs" in the current working directory
file_path = Path("figs",df_src['label'][0] + df_src['id'][0] + ".png").absolute()
print(file_path)
image.save(file_path, "PNG", quality=80)


/home/georg/git-projects/fastbook/bike-reloot/figs/not_stolen772596263.png


In [21]:
# Now iterate through the list 

for j in df_src.index:
    time.sleep(0.1)
    pass_image_file = io.BytesIO(requests.get(df_src['src'][j]).content)
    pass_image = Image.open(pass_image_file).convert("RGB")
    file_path = Path("figs",df_src['label'][j] +"-"+ df_src['src_id'][j] +"-"+ df_src['src_id'][j] + ".png").absolute()
    image.save(file_path, "PNG", quality=80)
    print("image with id "+df_src['src_id'][j]+" saved")
    print("URL: "+ df_src['src'][j])

image with id 348722016 saved
URL: https://cache.willhaben.at/mmo/3/772/596/263_-348722016.jpg
image with id 1845172890 saved
URL: https://cache.willhaben.at/mmo/3/772/596/263_-1845172890.jpg
image with id 263_906489552 saved
URL: https://cache.willhaben.at/mmo/3/772/596/263_906489552.jpg
image with id 186325535 saved
URL: https://cache.willhaben.at/mmo/3/772/596/263_-186325535.jpg


In [22]:
# Start the CNN

In [23]:
from fastai.vision.all import *

def is_stolen(x): return x.startswith('stolen')

print(is_stolen("stolen_asdf"))

path = Path("figs").absolute()

print(path)

dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_stolen, item_tfms=Resize(224))
dls
#learn = vision_learner(dls, resnet34, metrics=error_rate)
#learn.fine_tune(3)

True
/home/georg/git-projects/fastbook/bike-reloot/figs
